In [3]:
import json
import pandas as pd
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [2]:
from sentence_transformers import SentenceTransformer

In [4]:
embed_model = 'multi-qa-MiniLM-L6-cos-v1'

In [5]:
model = SentenceTransformer(embed_model)

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
with open('documents_id.json', 'rt') as f:
    doc = json.load(f)

In [7]:
doc[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c86136cf'}

In [8]:
for i in tqdm(doc):
    qt = i['question'] +' '+ i['text']
    i['questionv'] = model.encode(i['question'])
    i['textv'] = model.encode(i['text'])
    i['vqt'] = model.encode(qt)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [01:53<00:00,  8.33it/s]


In [9]:
esclient = Elasticsearch('http://localhost:9200')
es_index = 'search06'

In [16]:
esclient.info()

ObjectApiResponse({'name': '45678cc5d33a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'U0vK9DhWSDyWy73gbqxiVw', 'version': {'number': '8.9.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '8aa461beb06aa0417a231c345a1b8c38fb498a0d', 'build_date': '2023-07-19T14:43:58.555259655Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [18]:
es_setting = {
    'settings':{
        'number_of_shards':1,
        'number_of_replicas':0
    },
    'mappings':{
        'properties':{
            'text':{'type':'text'},
            'section':{'type':'text'},
            'question':{'type':'text'},
            'course':{'type':'keyword'},
            'id':{'type':'keyword'},
            'questionv':{'type':'dense_vector',
                        'dims':384,
                         'index': True,
                         'similarity': 'cosine'
                        },
            'textv':{'type':'dense_vector',
                        'dims':384,
                         'index': True,
                         'similarity': 'cosine'
                        },
            'vqt':{'type':'dense_vector',
                        'dims':384,
                         'index': True,
                         'similarity': 'cosine'
                        },
            
        }
    }
}

In [19]:
esclient.indices.create(index= es_index, body = es_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'search06'})

In [20]:
for i in tqdm(doc):
    esclient.index(index= es_index, document=i)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:21<00:00, 44.84it/s]


# hybrid search

In [27]:
query_sample = 'I just found the course. Can I join it?'
course = 'data-engineering-zoomcamp'
query_v = model.encode(query_sample)

In [35]:
knn_search = {
    'field': 'vqt',
    'query_vector' :query_v,
    'k':5,
    'num_candidates': 10000,
    'filter':{
                'term':{
                    'course': course
                }
            }
}

In [38]:
keyword_search = {
    'bool':{
        'must':{
            'multi_match':{
                'query': query,
                'fields': ['text', 'question', 'section'],
                'type':'best_fields'
            }
        },
        'filter':{
            'term':{
                'course': course
            }
        }
    }
}

In [39]:
response = esclient.search(index= es_index, query = keyword_search, knn = knn_search, size=5)

In [43]:
response['hits']['hits']

[{'_index': 'search06',
  '_id': '4HL60pEBk-9ILn4KLxqZ',
  '_score': 20.719679,
  '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
   'section': 'General course-related questions',
   'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp',
   'id': '03e9e63a',
   'questionv': [0.0030358789954334497,
    -0.0023871944285929203,
    0.0358816422522068,
    0.020998863503336906,
    -0.018282365053892136,
    0.06715094298124313,
    -0.10277319699525833,
    -0.11509548872709274,
    -0.06606756150722504,
    -0.004973309114575386,
    -0.002861772431060672,
    0.10543150454759598,
    -0.0008143176091834903,
    0.08418367058038712,
    0.02704712562263012,
    -0.031353775411844254,
    -0.05154323950409889,
    -0.049489956349134445,
    0.053